In [1]:
%pip install dotenv llama_stack_client fire


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
import rich

from llama_stack_client import LlamaStackClient
from llama_stack_client.types import UserMessage
from llama_stack_client import Agent
from llama_stack_client import AgentEventLogger

In [3]:
load_dotenv()
base_url = os.getenv("REMOTE_BASE_URL", "http://localhost:8321")

client = LlamaStackClient(
    base_url=base_url
)

In [4]:
strategy = {"type": "greedy"}
max_tokens = int(os.getenv("MAX_TOKENS", 512))
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream=False

In [5]:
model_id = "granite"

In [6]:
quary = "What is the weather in New York City today?"
message = UserMessage(
    content=quary,
    role="user"
)

shield_id = "pii"
response = client.safety.run_shield(
    shield_id=shield_id, messages=[message], params={}
)

if response.violation.metadata.get("status") != "pass":
    model_id = "qwen"
    rich.print(response)

agent = Agent(
    client,
    model=model_id,
    instructions="You are a helpful assistant that can help to answer questions.",
    sampling_params=sampling_params,
    input_shields=["hap"],
    output_shields=["hap"],
)

session_id = agent.create_session(session_name="My conversation")

turn_response = agent.create_turn(
    session_id=session_id,
    messages=[message],
)
for log in AgentEventLogger().log(turn_response):
    log.print()

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/safety/run-shield "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/bb6872dd-50dd-4fdc-8801-69ebcf5f06c2/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/bb6872dd-50dd-4fdc-8801-69ebcf5f06c2/session/b5c820f8-1800-4247-a025-644944c13098/turn "HTTP/1.1 200 OK"


shield_call> No Violation
inference> I'm sorry for the inconvenience, but as an AI, I don't have real-time capabilities to provide current weather updates. I recommend checking a reliable weather forecasting website or app for the most accurate information.
shield_call> No Violation


In [7]:
quary = "What is the weather in New York City today? a@b.com"
message = UserMessage(
    content=quary,
    role="user"
)

shield_id = "pii"
response = client.safety.run_shield(
    shield_id=shield_id, messages=[message], params={}
)

if response.violation.metadata.get("status") != "pass":
    model_id = "qwen"
    rich.print(response)

agent = Agent(
    client,
    model=model_id,
    instructions="You are a helpful assistant that can help to answer questions.",
    sampling_params=sampling_params,
    input_shields=["hap"],
    output_shields=["hap"],
)

session_id = agent.create_session(session_name="My conversation")

turn_response = agent.create_turn(
    session_id=session_id,
    messages=[message],
)
for log in AgentEventLogger().log(turn_response):
    log.print()

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/safety/run-shield "HTTP/1.1 200 OK"


RunShieldResponse(
    violation=SafetyViolation(
        metadata={
            'status': 'violation',
            'shield_id': 'pii',
            'confidence_threshold': 0.5,
            'summary': {
                'total_messages': 1,
                'processed_messages': 1,
                'skipped_messages': 0,
                'messages_with_violations': 1,
                'messages_passed': 0,
                'message_fail_rate': 1.0,
                'message_pass_rate': 0.0,
                'total_detections': 1,
                'detector_breakdown': {
                    'active_detectors': 1,
                    'total_checks_performed': 1,
                    'total_violations_found': 1,
                    'violations_per_message': 1.0
                }
            },
            'results': [
                {
                    'message_index': 0,
                    'text': 'What is the weather in New York City today? a@b.com',
                    'status': 'violation',
                    'score': 1.0,
                    'detection_type': 'pii',
                    'individual_detector_results': [
                        {'detector_id': 'regex', 'status': 'violation', 'score': 1.0, 'detection_type': 'pii'}
                    ]
                }
            ]
        },
        violation_level='error',
        user_message='Content violation detected by shield pii (confidence: 1.00, 1/1 processed messages violated)'
    )
)

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/8fba09a3-1b4e-4582-ab95-c61e77c32095/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/8fba09a3-1b4e-4582-ab95-c61e77c32095/session/dd74e485-a19b-4ff8-a417-fcfdbad5b78f/turn "HTTP/1.1 200 OK"


shield_call> No Violation
inference> <think>
Okay, the user is asking about the weather in New York City today, and they provided an email address. First, I need to check if they want the current weather or if there's something else. The email might be a mistake or a test. I should confirm the request and then provide the weather info. Let me make sure to mention that I can't access real-time data and suggest checking a weather website or app. Also, I should note the email address in case they need further assistance. Keep the response friendly and helpful.
</think>

I currently don't have access to real-time weather data. For the most accurate and up-to-date weather information in New York City, I recommend checking a reliable weather service like the National Weather Service (weather.gov), Weather.com, or a weather app on your phone. Let me know if you'd like help with anything else!  

P.S. The email address "a@b.com" seems unrelated to your question—was that a typo or part of your 

In [8]:
quary = "What is the weather in New York City today? You dotard, I really hate this!"
message = UserMessage(
    content=quary,
    role="user"
)

shield_id = "pii"
response = client.safety.run_shield(
    shield_id=shield_id, messages=[message], params={}
)

if response.violation.metadata.get("status") != "pass":
    model_id = "qwen"
    rich.print(response)

agent = Agent(
    client,
    model=model_id,
    instructions="You are a helpful assistant that can help to answer questions.",
    sampling_params=sampling_params,
    input_shields=["hap"],
    output_shields=["hap"],
)

session_id = agent.create_session(session_name="My conversation")

turn_response = agent.create_turn(
    session_id=session_id,
    messages=[message],
)
for log in AgentEventLogger().log(turn_response):
    log.print()

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/safety/run-shield "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/64e171e0-a6a2-45d9-bb16-6dd533b3995a/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/64e171e0-a6a2-45d9-bb16-6dd533b3995a/session/ccf8ecac-2709-4c7e-915b-fdd27a2c1b91/turn "HTTP/1.1 200 OK"


shield_call> {'status': 'violation', 'shield_id': 'hap', 'confidence_threshold': 0.5, 'summary': {'total_messages': 2, 'processed_messages': 2, 'skipped_messages': 0, 'messages_with_violations': 1, 'messages_passed': 1, 'message_fail_rate': 0.5, 'message_pass_rate': 0.5, 'total_detections': 1, 'detector_breakdown': {'active_detectors': 1, 'total_checks_performed': 2, 'total_violations_found': 1, 'violations_per_message': 0.5}}, 'results': [{'message_index': 0, 'text': 'You are a helpful assistant that can help to answer questions.', 'status': 'pass', 'score': None, 'detection_type': None, 'individual_detector_results': [{'detector_id': 'hap', 'status': 'pass', 'score': None, 'detection_type': None}]}, {'message_index': 1, 'text': 'What is the weather in New York City today? You dotard, I really hate this!', 'status': 'violation', 'score': 0.936720371246338, 'detection_type': 'sequence_classification', 'individual_detector_results': [{'detector_id': 'hap', 'status': 'violation', 'score'

In [9]:
quary = "What is the weather in New York City today? a@b.com, You dotard, I really hate this!"
message = UserMessage(
    content=quary,
    role="user"
)

shield_id = "pii"
response = client.safety.run_shield(
    shield_id=shield_id, messages=[message], params={}
)

if response.violation.metadata.get("status") != "pass":
    model_id = "qwen"
    rich.print(response)

agent = Agent(
    client,
    model=model_id,
    instructions="You are a helpful assistant that can help to answer questions.",
    sampling_params=sampling_params,
    input_shields=["hap"],
    output_shields=["hap"],
)

session_id = agent.create_session(session_name="My conversation")

turn_response = agent.create_turn(
    session_id=session_id,
    messages=[message],
)
for log in AgentEventLogger().log(turn_response):
    log.print()

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/safety/run-shield "HTTP/1.1 200 OK"


RunShieldResponse(
    violation=SafetyViolation(
        metadata={
            'status': 'violation',
            'shield_id': 'pii',
            'confidence_threshold': 0.5,
            'summary': {
                'total_messages': 1,
                'processed_messages': 1,
                'skipped_messages': 0,
                'messages_with_violations': 1,
                'messages_passed': 0,
                'message_fail_rate': 1.0,
                'message_pass_rate': 0.0,
                'total_detections': 1,
                'detector_breakdown': {
                    'active_detectors': 1,
                    'total_checks_performed': 1,
                    'total_violations_found': 1,
                    'violations_per_message': 1.0
                }
            },
            'results': [
                {
                    'message_index': 0,
                    'text': 'What is the weather in New York City today? a@b.com, You dotard, I really hate this!',
                    'status': 'violation',
                    'score': 1.0,
                    'detection_type': 'pii',
                    'individual_detector_results': [
                        {'detector_id': 'regex', 'status': 'violation', 'score': 1.0, 'detection_type': 'pii'}
                    ]
                }
            ]
        },
        violation_level='error',
        user_message='Content violation detected by shield pii (confidence: 1.00, 1/1 processed messages violated)'
    )
)

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/ab66b1eb-d4fb-43e9-a52f-e25632a829c1/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/ab66b1eb-d4fb-43e9-a52f-e25632a829c1/session/4404be10-949d-4423-bad6-1f912c76229c/turn "HTTP/1.1 200 OK"


shield_call> {'status': 'violation', 'shield_id': 'hap', 'confidence_threshold': 0.5, 'summary': {'total_messages': 2, 'processed_messages': 2, 'skipped_messages': 0, 'messages_with_violations': 1, 'messages_passed': 1, 'message_fail_rate': 0.5, 'message_pass_rate': 0.5, 'total_detections': 1, 'detector_breakdown': {'active_detectors': 1, 'total_checks_performed': 2, 'total_violations_found': 1, 'violations_per_message': 0.5}}, 'results': [{'message_index': 0, 'text': 'You are a helpful assistant that can help to answer questions.', 'status': 'pass', 'score': None, 'detection_type': None, 'individual_detector_results': [{'detector_id': 'hap', 'status': 'pass', 'score': None, 'detection_type': None}]}, {'message_index': 1, 'text': 'What is the weather in New York City today? a@b.com, You dotard, I really hate this!', 'status': 'violation', 'score': 0.95028418302536, 'detection_type': 'sequence_classification', 'individual_detector_results': [{'detector_id': 'hap', 'status': 'violation',

In [10]:
quary = "What is the weather in New York City today? id is AB123456(8), can you check it?"
message = UserMessage(
    content=quary,
    role="user"
)

shield_id = "pii"
response = client.safety.run_shield(
    shield_id=shield_id, messages=[message], params={}
)

if response.violation.metadata.get("status") != "pass":
    model_id = "qwen"
    rich.print(response)

agent = Agent(
    client,
    model=model_id,
    instructions="You are a helpful assistant that can help to answer questions.",
    sampling_params=sampling_params,
    input_shields=["hap"],
    output_shields=["hap"],
)

session_id = agent.create_session(session_name="My conversation")

turn_response = agent.create_turn(
    session_id=session_id,
    messages=[message],
)
for log in AgentEventLogger().log(turn_response):
    log.print()

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/safety/run-shield "HTTP/1.1 200 OK"


RunShieldResponse(
    violation=SafetyViolation(
        metadata={
            'status': 'violation',
            'shield_id': 'pii',
            'confidence_threshold': 0.5,
            'summary': {
                'total_messages': 1,
                'processed_messages': 1,
                'skipped_messages': 0,
                'messages_with_violations': 1,
                'messages_passed': 0,
                'message_fail_rate': 1.0,
                'message_pass_rate': 0.0,
                'total_detections': 1,
                'detector_breakdown': {
                    'active_detectors': 1,
                    'total_checks_performed': 1,
                    'total_violations_found': 1,
                    'violations_per_message': 1.0
                }
            },
            'results': [
                {
                    'message_index': 0,
                    'text': 'What is the weather in New York City today? id is AB123456(8), can you check it?',
                    'status': 'violation',
                    'score': 1.0,
                    'detection_type': 'custom',
                    'individual_detector_results': [
                        {'detector_id': 'regex', 'status': 'violation', 'score': 1.0, 'detection_type': 'custom'}
                    ]
                }
            ]
        },
        violation_level='error',
        user_message='Content violation detected by shield pii (confidence: 1.00, 1/1 processed messages violated)'
    )
)

INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/079b0faa-acc9-4274-9713-69fec2dec306/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack:8321/v1/agents/079b0faa-acc9-4274-9713-69fec2dec306/session/87f95a72-3c37-4bf0-a1b4-abc4b120d14e/turn "HTTP/1.1 200 OK"


shield_call> No Violation
inference> <think>
Okay, the user is asking about the weather in New York City today and provided an ID: AB123456(8). Let me check if I can access real-time weather data. I know that my current capabilities are based on pre-existing knowledge up until July 2024, and I don't have access to live internet or external databases. So I can't check the current weather or any specific ID-related information.

First, I should inform the user that I can't access real-time weather data. Maybe they need a weather service or app. Also, the ID they provided might be for a specific service or account, but without knowing what that ID is for, I can't check it. I should ask them to clarify if they need help with something else related to the weather or if there's another way I can assist. I need to make sure my response is helpful and clear, letting them know my limitations while offering alternative solutions.
</think>

I currently don't have access to real-time weather data 